In [17]:
from neo4j import GraphDatabase, basic_auth
import pandas as pd

In [6]:
class Neo4jConnection:

    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

In [7]:
conn = Neo4jConnection(uri="bolt://18.234.199.96:7687",
                       user="neo4j",
                       pwd="trip-classes-delights")

In [11]:
insert_query = """
CREATE
  (iphone14:Item {id: 1, name: 'iPhone 14', price: 900}),
  (iphone13:Item {id: 2, name: 'iPhone 13', price: 800}),
  (galaxys23:Item {id: 3, name: 'Galaxy s23', price: 850}),
  (watchseries8:Item {id: 4, name: 'Watch series 8', price: 400}),

  (cust_v:Customer {id: 1, name: 'Vadym'}),
  (cust_a:Customer {id: 2, name: 'Andriy'}),

  (order1:Order {id: 1, date: date("2022-03-01")}),
  (order2:Order {id: 2, date: date("2022-03-12")}),
  (order3:Order {id: 3, date: date("2019-03-23")}),

  (cust_v)-[:BOUGHT]->(order1),
  (cust_v)-[:BOUGHT]->(order2),
  (cust_a)-[:BOUGHT]->(order3),

  (order1) - [: CONTAINS]->(iphone14),
  (order1) - [: CONTAINS]->(iphone13),
  (order2) - [: CONTAINS]->(galaxys23),
  (order3) - [: CONTAINS]->(galaxys23),
  (order3) - [: CONTAINS]->(watchseries8),

  (cust_v) - [: VIEW]->(watchseries8),
  (cust_a) - [: VIEW]->(iphone14)
"""
conn.query(insert_query)

[]

In [68]:
insert_query2 = """
MATCH (c:Customer {id: 1}), (i:Item {id: 1})
CREATE (c) - [: VIEW] -> (i)
"""
conn.query(insert_query2)

[]

Знайти Items які входять в конкретний Order (за Order id)

In [22]:
query = """
MATCH (o:Order)-[:CONTAINS]-(Item)
WHERE o.id = 1
RETURN Item.name, Item.price"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,Item.name,Item.price
0,iPhone 13,800
1,iPhone 14,900


Підрахувати вартість конкретного Order

In [23]:
query = """
MATCH (o:Order)-[:CONTAINS]-(Item)
WHERE o.id = 1
RETURN sum(Item.price)"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,sum(Item.price)
0,1700


Знайти всі Orders конкретного Customer

In [24]:
query = """
MATCH (c:Customer)-[:BOUGHT]-(o:Order)
WHERE c.id = 1
RETURN o.id, o.date"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,o.id,o.date
0,2,2022-03-12
1,1,2022-03-01


Знайти всі Items куплені конкретним Customer (через його Orders)

In [26]:
query = """
MATCH (c:Customer)-[:BOUGHT]-(o:Order)-[:CONTAINS]-(i:Item)
WHERE c.id = 1
RETURN i.name, i.price"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,i.name,i.price
0,Galaxy s23,850
1,iPhone 13,800
2,iPhone 14,900


Знайти загальну кількість Items куплені конкретним Customer (через його Order)

In [27]:
query = """
MATCH (c:Customer)-[:BOUGHT]-(o:Order)-[:CONTAINS]-(i:Item)
WHERE c.id = 1
RETURN count(i) as items_cnt"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,items_cnt
0,3


Знайти для Customer на яку загальну суму він придбав товарів (через його Order)

In [28]:
query = """
MATCH (c:Customer)-[:BOUGHT]-(o:Order)-[:CONTAINS]-(i:Item)
WHERE c.id = 1
RETURN sum(i.price) as total_price"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,total_price
0,2550


Знайті скільки разів кожен товар був придбаний, відсортувати за цим значенням

In [32]:
query = """
MATCH (o:Order)-[:CONTAINS]-(i:Item)
RETURN i.name, count(i) as cnt
ORDER BY cnt DESC"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,i.name,cnt
0,Galaxy s23,2
1,iPhone 13,1
2,iPhone 14,1
3,Watch series 8,1


Знайти всі Items переглянуті (view) конкретним Customer

In [77]:
query = """
MATCH (c:Customer)-[:VIEW]-(i:Item)
WHERE c.id = 1
RETURN i.name, i.price"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,i.name,i.price
0,iPhone 14,900
1,Watch series 8,400


Знайти інші Items що купувались разом з конкретним Item (тобто всі Items що входять до Order-s разом з даними Item)

In [47]:
query = """
MATCH (o:Order)-[:CONTAINS]-(i:Item)
WHERE EXISTS {MATCH (o)-[:CONTAINS]-(:Item {id: 3})}
    AND i.id <> 3
RETURN i.name"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,i.name
0,Watch series 8


Знайти Customers які купили даний конкретний Item

In [59]:
query = """
MATCH (c:Customer)-[:BOUGHT]-(o:Order)-[:CONTAINS]-(i:Item {id: 3})
RETURN DISTINCT c.name"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,c.name
0,Andriy
1,Vadym


Знайти для певного Customer(а) товари, які він переглядав, але не купив

In [76]:
query = """
MATCH (c:Customer {id: 1})-[:VIEW]-(i:Item)
WHERE NOT EXISTS {MATCH (c)-[:BOUGHT]-(o:Order)-[:CONTAINS]-(i)}
RETURN i.name, i.price"""
df = pd.DataFrame([dict(_) for _ in conn.query(query)])
df

,i.name,i.price
0,Watch series 8,400
